In [ ]:
# Import required libraries and utility functions for the experiment
import pandas as pd
import json
from utils.constants import OBJECTIVES_SUDOKU
from utils.utils import load_sudokus_from_json, create_folders
from utils.utils_classes import Oracle
from utils.utils_training import PreferenceElicitationFramework
from utils.utils_classes import Human
from utils.utils_sudoku import generate_steps_sudoku
import ast
import numpy as np
from tqdm import tqdm
from utils.utils import count_and_remove_matching_hints

In [ ]:
# Set experiment parameters: learning rate, normalization type, and step selection strategy
lr = 1
normalization = 2
steps = 'SMUS'

In [ ]:
# Prompt user for username and print confirmation
username = input("Enter your username: ")
if not username.strip():
    print("Username is required!")
else:
    print(f"Username entered: {username}")

# Warmup OF 5 Pairs
Green indicates the cell that we want to explain.\n
Yellow indicates a filled cell that is used in the explanation
Blue highligh one constraint (row, column, block) that is used for the explanation

In [ ]:
sudokus = load_sudokus_from_json()

steps_sudokus = []
for index in range(15):
    steps_single_sudoku = []
    df_steps = pd.read_csv(f'data/gt_sudoku/SMUS/sudoku_user_SMUS_sudoku_{index}.csv',index_col=False)
    for index, row in df_steps.iterrows():
        grid = np.fromstring(row['grid'].replace("[", "").replace("]", ""), sep=" ").astype(int).reshape(9, 9)
        r,c = ast.literal_eval(row['hint_derived'])[1][:2]
        steps_single_sudoku.append([grid,[r,c]])
    steps_sudokus.append(steps_single_sudoku)


oracle = Human()
output_location_machop = f'results/warmup'
output_location_baseline = f'results/warmup/'
user = 0
df_steps_evaluation = ''
initial_weights = dict.fromkeys(OBJECTIVES_SUDOKU, 1)


pef = PreferenceElicitationFramework(logic_puzzles_set=sudokus[5:], oracle=oracle, no_oracle=user,
                                     initial_weights=initial_weights, instance_evaluation=sudokus[30],
                                     df_steps_evaluation=df_steps_evaluation, output_location=output_location_machop,
                                     time_eval=150,max_data=5,
                                     normalized=normalization, batch_size=1, frozen_steps=steps_sudokus[5:],
                                     lr=lr, type_diversification='MACHOP',exploration_root=1/2)
learned_weights  = pef.start()

# Start Real Experiment
## Training by labelling 50 pairs

In [ ]:
# Reload sudoku puzzles and prepare steps for real experiment
sudokus = load_sudokus_from_json()

steps_sudokus = []
for index in range(15):
    steps_single_sudoku = []
    # Read ground truth steps for each sudoku from CSV
    df_steps = pd.read_csv(f'data/gt_sudoku/SMUS/sudoku_user_SMUS_sudoku_{index}.csv',index_col=False)
    for index, row in df_steps.iterrows():
        # Parse grid and hint coordinates from CSV row
        grid = np.fromstring(row['grid'].replace("[", "").replace("]", ""), sep=" ").astype(int).reshape(9, 9)
        r,c = ast.literal_eval(row['hint_derived'])[1][:2]
        steps_single_sudoku.append([grid,[r,c]])
    steps_sudokus.append(steps_single_sudoku)

# Set initial weights and output locations for real experiment
weights = dict.fromkeys(OBJECTIVES_SUDOKU, 1)

oracle = Human()
output_location_machop = f'results_AAAI/sudoku/real_case_{username}_{lr}_MACHOP/'
output_location_baseline = f'results_AAAI/sudoku/real_case_{username}_{lr}_baseline/'

## Training by labelling 50 pairs 

In [ ]:
# Run preference elicitation framework for baseline strategy (50 pairs)
user = 0
df_steps_evaluation = ''
initial_weights = dict.fromkeys(OBJECTIVES_SUDOKU, 1)
pef = PreferenceElicitationFramework(logic_puzzles_set=sudokus, oracle=oracle, no_oracle=user,
                                     initial_weights=initial_weights, instance_evaluation=sudokus[30],
                                     df_steps_evaluation=df_steps_evaluation, output_location=output_location_baseline,
                                     time_eval=150,max_data=50,
                                     normalized=normalization, batch_size=1, frozen_steps=steps_sudokus,
                                     lr=lr, type_diversification='baseline', exploration_root=1)
learned_weights  = pef.start()

In [ ]:
# Function to retrieve weights and normalization values for a given step
def get_weights_norm_steps(step):
    # Read weights and normalization for MACHOP strategy
    df_weights_machop = pd.read_csv(f'{output_location_machop}weights.csv')
    df_norm_machop = pd.read_csv(f'{output_location_machop}norm.csv')

    weights_column = df_weights_machop['weights']
    last_value_str = weights_column.iloc[step-1]
    weights_machop = ast.literal_eval(last_value_str)

    weights_column = df_norm_machop['norm']
    last_value_str = weights_column.iloc[step-1]
    norm_machop = ast.literal_eval(last_value_str)

    # Read weights and normalization for baseline strategy
    df_weights_baseline = pd.read_csv(f'{output_location_baseline}weights.csv')
    df_norm_baseline = pd.read_csv(f'{output_location_baseline}norm.csv')

    weights_column = df_weights_baseline['weights']
    last_value_str = weights_column.iloc[step-1]
    weights_baseline = ast.literal_eval(last_value_str)

    weights_column = df_norm_baseline['norm']
    last_value_str = weights_column.iloc[step-1]
    norm_baseline = ast.literal_eval(last_value_str)

    return weights_machop,norm_machop,weights_baseline,norm_baseline

# Evaluation - Generation of the steps

In [ ]:
# Generate predicted steps using MACHOP weights for steps 10, 30, 50
for step in [10,30,50]:
    weights_machop,norm_machop,weights_baseline,norm_baseline = get_weights_norm_steps(step)
    df_steps_evaluation = pd.read_csv('data/gt_sudoku/sudoku_gt_SMUS_sudoku_30.csv')
    predicted = []
    for index, row in tqdm(df_steps_evaluation.iterrows(), total=len(df_steps_evaluation)):
        # Prepare sudoku grid and hint for evaluation
        sudoku_eval = row['grid'].replace(' ', ', ')
        sudoku_eval = np.array(eval(sudoku_eval))
        parsed_list = eval(row['hint_derived'])
        explained_row = parsed_list[1][0]
        explained_col = parsed_list[1][1]
        # Generate explanation using learnt MACHOP weights
        exp,_,_ = generate_steps_sudoku(to_return=True, grid=sudoku_eval,
                                        feature_weights=weights_machop,no_user=1,
                                        sequential_sudoku=-1,normalization=norm_machop,
                                        is_tqdm=False)
        predicted.append(exp)

    # Save predicted steps to CSV
    df_predicted_steps = pd.DataFrame(predicted)
    df_predicted_steps.to_csv(f'results_AAAI/sudoku/real_case_{username}_{lr}_MACHOP/machop_SMUS_{step}.csv', index=False)

In [ ]:
# Generate predicted steps using baseline weights for steps 10, 30, 50
for step in [10,30,50]:
    weights_machop,norm_machop,weights_baseline,norm_baseline = get_weights_norm_steps(step)
    df_steps_evaluation = pd.read_csv('data/gt_sudoku/sudoku_gt_SMUS_sudoku_30.csv')
    predicted = []
    for index, row in tqdm(df_steps_evaluation.iterrows(), total=len(df_steps_evaluation)):
        # Prepare sudoku grid and hint for evaluation
        sudoku_eval = row['grid'].replace(' ', ', ')
        sudoku_eval = np.array(eval(sudoku_eval))
        parsed_list = eval(row['hint_derived'])
        explained_row = parsed_list[1][0]
        explained_col = parsed_list[1][1]
        # Generate explanation using learnt baseline weights
        exp,_,_ = generate_steps_sudoku(to_return=True, grid=sudoku_eval,
                                        feature_weights=weights_baseline,no_user=1,
                                        sequential_sudoku=-1,normalization=norm_baseline,
                                        is_tqdm=False)
        predicted.append(exp)

    # Save predicted steps to CSV
    df_predicted_steps = pd.DataFrame(predicted)
    df_predicted_steps.to_csv(f'results_AAAI/sudoku/real_case_{username}_{lr}_baseline/baseline_SMUS_{step}.csv', index=False)

In [ ]:
# Load ground truth and predicted steps for evaluation and comparison
dict_df = {}
for step in [10,30,50]:
    df_smus = pd.read_csv('data/gt_sudoku/sudoku_gt_SMUS_sudoku_30.csv')
    df_learnt_baseline = pd.read_csv(f'results_AAAI/sudoku/real_case_{username}_{lr}_baseline/baseline_SMUS_{step}.csv')
    df_learnt_machop = pd.read_csv(f'results_AAAI/sudoku/real_case_{username}_{lr}_MACHOP/machop_SMUS_{step}.csv')
    dict_df[step] = [df_smus, df_learnt_baseline, df_learnt_machop]

In [ ]:
# Compare learnt preferences (MACHOP) with SES by labelling and saving results
for step in [10,30,50]:
    df_smus,df_learnt_baseline,df_learnt_machop = dict_df[step]
    # Remove matching hints and get cleaned dataframes
    ties, df_smus_cleaned, df_learnt_machop_cleaned = count_and_remove_matching_hints(df_smus.copy(), df_learnt_machop.copy())
    res = [df_smus_cleaned,df_learnt_machop_cleaned]
    df_hand, df_learnt = res

    # Shuffle rows for unbiased labelling
    shuffled_indices = np.random.permutation(len(df_hand))
    df_hand_shuffled = df_hand.iloc[shuffled_indices].reset_index(drop=True)
    df_learnt_shuffled = df_learnt.iloc[shuffled_indices].reset_index(drop=True)
    res_shuffled = [df_hand_shuffled, df_learnt_shuffled]

    # Label explanations using Human oracle
    oracle = Human()
    res = oracle.label(res_shuffled,evaluation=True)
    res['identical'] = ties
    create_folders(f'results_AAAI/sudoku/real_case_{username}/')
    # Save labelling results to JSON
    with open(f'results_AAAI/sudoku/real_case_{username}/SMUS_machop_{step}.json', "w") as json_file:
        json.dump(res, json_file, indent=4)

In [ ]:
# Compare learnt preferences (baseline) with SES by labelling and saving results
for step in [10,30,50]:
    df_smus,df_learnt_baseline,df_learnt_machop = dict_df[step]
    # Remove matching hints and get cleaned dataframes
    ties, df_smus_cleaned, df_learnt_baseline_cleaned = count_and_remove_matching_hints(df_smus.copy(), df_learnt_baseline.copy())
    res = [df_smus_cleaned,df_learnt_baseline_cleaned]
    df_hand, df_learnt = res

    # Shuffle rows for unbiased labelling
    shuffled_indices = np.random.permutation(len(df_hand))
    df_hand_shuffled = df_hand.iloc[shuffled_indices].reset_index(drop=True)
    df_learnt_shuffled = df_learnt.iloc[shuffled_indices].reset_index(drop=True)
    res_shuffled = [df_hand_shuffled, df_learnt_shuffled]

    # Label explanations using Human oracle
    oracle = Human()
    res = oracle.label(res_shuffled,evaluation=True)
    res['identical'] = ties
    create_folders(f'results_AAAI/sudoku/real_case_{username}/')
    # Save labelling results to JSON
    with open(f'results_AAAI/sudoku/real_case_{username}/SMUS_baseline_{step}.json', "w") as json_file:
        json.dump(res, json_file, indent=4)